In [ ]:
# imports
from langchain.messages import AnyMessage
from typing_extensions import TypedDict, List, Annotated, Optional, Literal
from langchain.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import END, StateGraph, START
from langchain_tavily import TavilySearch
import os
import operator
from pydantic import BaseModel, Field
import heapq

from config import config_rag
from hybrid_database import hybrid_search
from hybrid_database import data_preprocessing
from FlagEmbedding import FlagLLMReranker
from langchain_openai import ChatOpenAI

In [ ]:
def get_models(config):
    
    database, embedding_model = data_preprocessing(config)
    print("loaded datbase and embedding_model")

    rerank_model = FlagLLMReranker('BAAI/bge-reranker-v2-gemma', use_fp16=False, devices=config["device"])
    print("loaded rerank model")

    llm_model = ChatOpenAI(
        model_name="Qwen/Qwen3-8B-AWQ",
        base_url="http://localhost:8000/v1",
        api_key="token-abc123",
        model_kwargs={
            "extra_body": {
                "guided_decoding_backend": "xgrammar" 
            }
        }
    )
    print("loaded llm model")
    print("All models loaded!!")
    return database, embedding_model, rerank_model, llm_model

In [ ]:
# get models, embeddings and index
config = config_rag()

database, embedding_model, rerank_model, llm_model = get_models(config)

In [ ]:
# tavilly client
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = config["tavilly_api_key"]

web_tool = TavilySearch(max_results=5, topic="general", include_images=False, )
docs = web_tool.invoke({"query": "Who is persona 5 main character?"})

print(docs)
web_results = "\n".join([d["content"] for d in docs["results"]])
print(web_results)

{'query': 'Who is persona 5 main character?', 'response_time': 0.88, 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://en.wikipedia.org/wiki/Joker_(Persona)', 'title': 'Joker (Persona) - Wikipedia', 'content': '# Joker (*Persona*). **Ren Amamiya**, also better known by his codename **Joker**, is the main protagonist of *Persona 5*, a 2016 role-playing video game owned by Sega subsidiary Atlus. Ultimate* by Nintendo, and the kart racing game *Sonic Racing: CrossWorlds* by Sega and Sonic Team He was designed by *Persona* series artist Shigenori Soejima, and is voiced in Japanese by Jun Fukuyama and in English by Xander Mobus. Joker is a male teenage character created by artist Shigenori Soejima for the 2016 video game *Persona 5*. Joker\'s initial Persona was originally the German demon Mephistopheles, but it was changed to Arsène since it better fit the game\'s themes. Joker was introduced in *Persona 5* as the game\'s main playable character and ha

In [ ]:
# define tools


# # Augment the LLM with tools
# tools = []
# tools_by_name = {tool.name: tool for tool in tools}
# model_with_tools = model.bind_tools(tools)

In [ ]:
question = ""
documents = ""

### LLMS

In [ ]:
# router node
class RouteDecision(BaseModel):
    """Routes the user query to the appropriate data source."""
    reasoning: str = Field(
        ..., 
        description="Briefly explain what the user is asking and check if it matches the explicit database topics."
    )
    is_in_domain: bool = Field(
        ..., 
        description="True ONLY if the query is strictly about the specific topics in our database (e.g., vLLM, CRAG, Self-RAG). False for general ML topics like Federated Learning, Vision, etc."
    )
    route: Literal["vectorstore", "websearch", "chitchat"] = Field(
        ..., 
        description="Choose 'vectorstore' if is_in_domain is True. Choose 'websearch' if is_in_domain is False or for current events. Choose 'chitchat' for greetings."
    )

router_node_prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert routing assistant for a highly specialized AI Engineering knowledge base. \nYour job is to analyze the user\'s query and route it to the correct destination.\n\nCRITICAL CONTEXT: Our \'vectorstore\' is NOT a general Machine Learning database. It ONLY contains 15 specific research papers. \nThe explicit topics covered in the vectorstore are:\n- LLM Architecture & Serving (vLLM, Transformers, PagedAttention)\n- Advanced RAG Methodologies (CRAG, Self-RAG, Adaptive-RAG, Vector Databases)\n- Model Building from Scratch (ConvNeXt)\n\nRouting Rules:\n1. Route to \'vectorstore\' ONLY if the query is explicitly related to the specific topics/papers listed above.\n2. Route to \'websearch\' if the user asks about General ML/AI topics NOT listed above (e.g., Federated Learning, Reinforcement Learning, CNNs, Generative Adversarial Networks).\n3. Route to \'websearch\' if the user asks about real-world current events, news, or live data.\n4. Route to \'chitchat\' if the query is a simple conversational greeting or compliment."),
        ("human", "Route this query: {question}"),
    ],
    input_variables = ["question"],
)

router_node_llm = router_node_prompt | llm_model.with_structured_output(RouteDecision, method="json_schema", strict=True)

result = router_node_llm.invoke({"question": question})

print(type(result))
print(result.route)

In [ ]:
# rewrite node
class RewrittenQuery(BaseModel):
    """The optimized search query for the vector database."""
    reasoning: str = Field(..., description="Briefly explain what keywords were extracted or expanded.")
    query: str = Field(..., description="The highly optimized, keyword-dense search query.")


rewrite_node_prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert Search Query Optimizer. \nThe user previously searched a vector database, but the retrieval failed to find relevant context.\n\nYour task is to rewrite the query to be highly effective for semantic search.\nStrip away conversational filler (e.g., \"Can you tell me about...\"). Extract core technical keywords and expand known AI acronyms to maximize the chance of a database match."),
        ("human", "Original failing query: {question}"),
    ],
    input_variables = ["question"],
)

rewrite_node_llm = rewrite_node_prompt | llm_model.with_structured_output(RewrittenQuery, method="json_schema", strict=True)


result = rewrite_node_llm.invoke({"question": question})

print(type(result))
print(result.query)

In [ ]:
# context checking before generation
class ContextGap(BaseModel):
    """Evaluates if there is missing foundational knowledge in the retrieved documents."""
    reasoning: str = Field(..., description="Explain if any concepts mentioned in the documents are required to answer the prompt, but are not defined.")
    has_gap: bool = Field(..., description="True if a critical concept/acronym is missing its definition. False if the context is complete.")
    missing_concept_query: str = Field(default="", description="If has_gap is True, provide a 2-4 word search query to find the missing concept. If False, leave empty.")

context_check_node_prompt = ChatPromptTemplate(
    [
        ("system", "You are a Context Gap Analyzer. \nYour job is to read the user\'s question and the currently retrieved documents to determine if a \"Multi-Hop\" search is required.\n\nLook for unexplained technical acronyms or foundational concepts that are mentioned in the text as part of the answer, but are not actually explained. \nIf explaining that concept is necessary to fully answer the user\'s question, flag a context gap and provide a short search query to retrieve that specific definition."),
        ("human", "User Question: {question}\n\nCurrently Retrieved Documents:\n{documents}"),
    ],
    input_variables = ["question", "documents"],
)

context_check_node_llm = context_check_node_prompt | llm_model.with_structured_output(ContextGap, method="json_schema", strict=True)

result = context_check_node_llm.invoke({"question": question, "documents": documents})

print(type(result))
print(result.has_gap)
print(result.missing_concept_query)

In [ ]:
# generate node
generate_node_prompt = ChatPromptTemplate(
    [
        ("system", "You are a highly technical AI Research Assistant.\nAnswer the user\'s question strictly using the provided context documents.\n\nRules:\n1. If the answer is not contained within the context documents, state: \"I cannot answer this based on the provided context.\"\n2. Do not use outside knowledge or hallucinate facts.\n3. Keep the answer concise, professional, and directly address the user\'s core question.\n4. When applicable, briefly cite the document or paper title you are referencing."),
        ("human", "Context Documents:\n{documents}\n\n---------------------\nUser Question: {question}"),
    ],
    input_variables = ["documents", "question"],
)

generate_node_llm = generate_node_prompt | llm_model | StrOutputParser()

result = generate_node_llm.invoke({"question": question, "documents": documents})

print(result)
generation = ""

In [ ]:
# hallucination checker
class HallucinationScore(BaseModel):
    """Checks if the generated answer contains hallucinations."""
    reasoning: str = Field(..., description="Briefly compare the generated facts against the source documents.")
    is_grounded: bool = Field(..., description="True if all facts in the generation are present in the documents. False if any outside info was added.")

hallucination_check_node_prompt = ChatPromptTemplate(
    [
        ("system", "You are a strict grading assistant executing a Self-RAG critique.\nYour task is to evaluate if an AI-generated answer is completely grounded in the provided source documents.\n\nA generation is ungrounded (hallucinated) if it includes statistics, methodology names, or factual claims that do not appear anywhere in the source text."),
        ("human", "Source Documents:\n{documents}\n\n---------------------\nAI Generation:\n{generation}"),
    ],
    input_variables = ["documents", "generation"],
)

hallucination_check_node_llm = hallucination_check_node_prompt | llm_model.with_structured_output(HallucinationScore, method="json_schema", strict=True)

result = hallucination_check_node_llm.invoke({"documents": documents, "generation": generation})

print(type(result))
print(result.is_grounded)

In [ ]:
# relevance checker
class RelevanceScore(BaseModel):
    """Checks if the generated answer actually addresses the user's question."""
    reasoning: str = Field(..., description="Explain how the generated text does or does not answer the core premise of the user's question.")
    is_relevant: bool = Field(..., description="True if the answer directly addresses the question. False if it is evasive or off-topic.")

relevance_check_node_prompt = ChatPromptTemplate(
    [
        ("system", "You are a strict grading assistant. \nYour task is to evaluate if an AI-generated answer successfully and directly addresses the user\'s original question.\n\nFail the generation if it just summarizes the documents without answering the specific user prompt, or if it is overly evasive."),
        ("human", "User Question: {question}\n\n---------------------\nAI Generation: {generation}"),
    ],
    input_variables = ["question", "generation"],
)

relevance_check_node_llm = relevance_check_node_prompt | llm_model.with_structured_output(RelevanceScore, method="json_schema", strict=True)

result = relevance_check_node_llm.invoke({"question": question, "generation": generation})

print(type(result))
print(result.is_relevant)

In [ ]:
# graph state
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
        loop_count: number to track loops
        gen_retries: number of generates for hallucinations
    """
    question: str
    generation: str
    documents: list
    missing_concept_query: str
    loop_count: int
    gen_retries: int

In [ ]:
# Nodes
def retrieve_and_rerank(state):
    """
    Retrieves documents from the index and immediately reranks them, 
    applying a score filter and keeping only the top k.
    
    Args:
        state (dict): Current graph state

    Returns:
        dict: The updated state containing strictly the highly relevant documents.
    """
    print("\n---RETRIEVE & RERANK---")
    
    question = state["question"]
    loop_count = state.get("loop_count", 0)

    print(f"Retrieving for question: {question}")
    raw_docs = hybrid_search(database, embedding_model, question, sparse_weight=0.7, dense_weight=1, limit=20)

    if not raw_docs:
        print("No documents found in database.")
        return {"documents": [], "question": question, "loop_count": loop_count}

    print("Reranking retrieved documents...")
    question_and_docs = [[question, doc["text"]] for doc in raw_docs]
    scores = rerank_model.compute_score(question_and_docs, normalize=True, batch_size=4, max_length=1024)
    
    filtered_pairs = [(doc, score) for doc, score in zip(raw_docs, scores) if score > 0.5]
    reranked_docs = heapq.nlargest(5, filtered_pairs, key=lambda x: x[1])
    
    final_documents = [doc for doc, score in reranked_docs]
    
    print(f"Milvus found {len(raw_docs)} docs. Reranker kept {len(final_documents)} docs > 0.5 score.")
    
    return {"documents": final_documents, "question": question, "loop_count": loop_count}       

def generate(state):
    """
    generate answer for query
    
    Args:
        state (dict): Current graph state

    Returns:
        state (dict): retrieved documents added to the state
    """
    print("\n---GENERATE---")

    question = state["question"]
    documents = state["documents"]
    loop_count = state["loop_count"]
    gen_retries = state.get("gen_retries", 0)

    result = generate_node_llm.invoke({"question": question, "documents": documents})
    generation = result

    gen_retries += 1
    
    return {"documents": documents, "question": question, "generation": generation, "loop_count": loop_count, "gen_retries": gen_retries}

def rewrite_query(state):
    """
    rewrite query for better question
    
    Args:
        state (dict): Current graph state

    Returns:
        state (dict): retrieved documents added to the state
    """
    print("\n---REWRITE_QUERY---")

    question = state["question"]
    documents = state["documents"]
    loop_count = state["loop_count"]
    loop_count += 1

    result = rewrite_node_llm.invoke({"question": question})
    rewritten_query = result.query
    
    return {"documents": documents, "question": rewritten_query, "loop_count": loop_count}

def web_search(state):
    """
    web search for query to find relevant context
    
    Args:
        state (dict): Current graph state

    Returns:
        state (dict): retrieved documents added to the state
    """
    print("\n---WEB SEARCH---")
    question = state["question"]

    docs = web_tool.invoke({"query": f"{question}"})

    print(docs)
    web_results = [d["content"] for d in docs["results"]]
    print(web_results)
    
    return {"documents": web_results, "question": question}

def chitchat_node(state):
    """Handles simple greetings without searching the database."""
    print("\n---CHITCHAT---")
    question = state["question"]
    
    result = generate_node_llm.invoke(f"The user said: '{question}'. Respond politely and concisely.")
    
    return {"generation": result, "question": question}

In [ ]:
# Edges

def query_router(state):
    """
    [Adaptive RAG]: routes the query to vectorbase or simple llm call or more
    
    Args:
        state (dict): Current graph state

    Returns:
        state (dict): retrieved documents added to the state
    """
    print("\n---Query Router--")
    question = state["question"]
    result = router_node_llm.invoke({"question": question})
    print(result.reasoning)
    if result.route == "vectorstore":
        print("---Route question to RAG")
        return "vectorstore"
    elif result.route == "websearch":
        print("---Route question to Websearch")
        return "websearch"
    elif result.route == "chitchat":
        print("---Route question to casual LLM")
        return "chitchat"

def rewrite_router(state):
    """
    [Adaptive RAG]: routes the query to vectorbase or simple llm call or more
    
    Args:
        state (dict): Current graph state

    Returns:
        state (dict): retrieved documents added to the state
    """
    loop_count = state["loop_count"]
    if loop_count > 3:
        return "web_search"
    else:
        return "retrieve"

def hallucinations_and_relevence_router(state):
    """
    [Adaptive RAG]: routes the query to vectorbase or simple llm call or more
    
    Args:
        state (dict): Current graph state

    Returns:
        state (dict): retrieved documents added to the state
    """
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    gen_retries = state["gen_retries"]

    # hallucination check
    print("\n---Checking hallucinations---")
    result = hallucination_check_node_llm.invoke({"documents": documents, "generation": generation})
    print(result.reasoning)
    if result.is_grounded:
        print("Hallucination Check passed")
        print("\n---Check relevance---")
        result = relevance_check_node_llm.invoke({"question": question, "generation": generation})
        print(result.reasoning)
        if result.is_relevant:
            return "all_pass"
    
    if gen_retries >= 2:
        return "rewrite_query"
    else:
        return "generate"
    
def post_retrieve_router(state):
    """
    Checks if any documents survived the reranker cutoff.
    """
    print("\n---POST RETRIEVE ROUTER---")
    documents = state.get("documents", [])
    
    if len(documents) == 0:
        print("---No relevant documents found. Routing to REWRITE---")
        return "rewrite"
    else:
        print(f"---Found {len(documents)} relevant documents. Routing to GENERATE---")
        return "generate"

In [ ]:
# compile graph
workflow = StateGraph(GraphState)

# 2. Add all the Nodes (The Workers)
workflow.add_node("retrieve_and_rerank", retrieve_and_rerank)
workflow.add_node("generate", generate)
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("web_search", web_search)
workflow.add_node("chitchat", chitchat_node)


workflow.add_conditional_edges(
    START,
    query_router,
    {
        "vectorstore": "retrieve_and_rerank",
        "websearch": "web_search",
        "chitchat": "chitchat"
    }
)

workflow.add_conditional_edges(
    "retrieve_and_rerank",
    post_retrieve_router,
    {
        "generate": "generate",
        "rewrite": "rewrite_query"
    }
)

workflow.add_conditional_edges(
    "rewrite_query",
    rewrite_router,
    {
        "retrieve": "retrieve_and_rerank",
        "web_search": "web_search"
    }
)

workflow.add_edge("web_search", "generate")

workflow.add_conditional_edges(
    "generate",
    hallucinations_and_relevence_router,
    {
        "all_pass": END,
        "generate": "generate",          
        "rewrite_query": "rewrite_query" 
    }
)

# Chitchat just ends the graph
workflow.add_edge("chitchat", END)

# 4. Compile into a runnable application
app = workflow.compile()

print("Graph compiled successfully!")

In [ ]:
from pprint import pprint

inputs = {"question": "What is the AlphaCodium paper about?"}

for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])